# Lab 8: P-Tuning v2 - Deep Prompt Tuning for Diverse Tasks
---
## Notebook 2: The Training Process

**Goal:** In this notebook, you will fine-tune a model using **P-Tuning v2** on both understanding and generation tasks. P-Tuning v2 applies deep prompts to every transformer layer, providing universal effectiveness across tasks and scales.


### Step 1: Load Dataset and Preprocess

We will use both classification and generation tasks to demonstrate P-Tuning v2's versatility. Let's start with the GLUE CoLA dataset for classification.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load dataset - CoLA for linguistic acceptability classification
dataset = load_dataset("glue", "cola", split="train[:1000]")
dataset = dataset.train_test_split(test_size=0.1)

# Load tokenizer
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Preprocessing function
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)

# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets.set_format("torch")

print("✅ Dataset loaded and preprocessed.")
print(f"Train samples: {len(tokenized_datasets['train'])}")
print(f"Test samples: {len(tokenized_datasets['test'])}")
print(tokenized_datasets["train"][0].keys())


### Step 2: Load the Base Model

Next, we load the BERT model for sequence classification.


In [ ]:
# Load base model
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, 
    num_labels=2  # Binary classification for CoLA
)

print("✅ Base BERT model loaded.")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")


### Step 3: Configure P-Tuning v2

Now we configure P-Tuning v2. Unlike P-Tuning v1, P-Tuning v2 applies prompts to every transformer layer, providing deeper integration and better performance.

#### Key Differences from P-Tuning v1:
-   **Deep Prompts**: Prompts are added to every transformer layer, not just the input.
-   **No MLP Encoder**: P-Tuning v2 uses direct prompt optimization without MLP.
-   **Universal Effectiveness**: Works well across various tasks and model scales.
-   **Simplified Design**: Easier to implement and more stable training.


In [ ]:
from peft import get_peft_model, PromptTuningConfig, TaskType

# --- P-Tuning v2 Configuration ---
# P-Tuning v2 uses PromptTuningConfig with specific settings for deep prompts
ptuning_v2_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,           # Sequence classification task
    prompt_tuning_init="RANDOM",          # Random initialization
    num_virtual_tokens=100,               # Longer prompt sequence for P-Tuning v2
    prompt_tuning_init_text=None,         # No text initialization
    tokenizer_name_or_path=model_checkpoint
)

# --- Create PeftModel ---
peft_model = get_peft_model(model, ptuning_v2_config)

# --- Print Trainable Parameters ---
peft_model.print_trainable_parameters()


### Step 4: Set Up Training

The final step is to configure and run the training process using the `transformers.Trainer`.


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef

# --- Evaluation Function ---
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted"),
        "matthews_correlation": matthews_corrcoef(labels, predictions)
    }

# --- Training Arguments ---
training_args = TrainingArguments(
    output_dir="./bert-ptuning-v2-cola",
    auto_find_batch_size=True,
    learning_rate=1e-3,                    # Higher learning rate for PEFT
    num_train_epochs=5,
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="matthews_correlation",
    greater_is_better=True,
)

# --- Data Collator ---
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# --- Create Trainer ---
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# --- Start Training ---
print("🚀 Starting training with P-Tuning v2...")
trainer.train()
print("✅ Training complete!")
